In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from tensorflow.keras.callbacks import TensorBoard
log_folder = './tf/rbf_logs'
callbacks = [TensorBoard(log_dir=log_folder,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=1,
                         embeddings_freq=1)]

2023-03-29 19:52:37.578309: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 19:52:38.509908: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2023-03-29 19:52:38.510035: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2023-03-29 19:52:38.510049: W tensorflow/com

In [4]:
# Load data
X, y = load_iris(return_X_y=True)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Find centers using K-means clustering
num_centers = 3
kmeans = KMeans(n_clusters=num_centers, random_state=42).fit(X_train)
centers = kmeans.cluster_centers_


/home/namachu/.local/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [5]:
# Define RBF layer
class RBFLayer(keras.layers.Layer):
    def __init__(self, num_centers):
        super(RBFLayer, self).__init__()
        self.num_centers = num_centers
        self.centers = self.add_weight(name='centers', shape=(self.num_centers, X_train.shape[1]), initializer='uniform', trainable=True)
        self.widths = self.add_weight(name='widths', shape=(self.num_centers,), initializer='ones', trainable=True)
        self.linear = layers.Dense(units=1, activation=None)

    def radial(self, X):
        X = tf.expand_dims(X, axis=1)
        centers = tf.expand_dims(self.centers, axis=0)
        dist = tf.reduce_sum(tf.square(X - centers), axis=-1)
        return tf.exp(-dist / (2 * tf.square(self.widths)))
    
    def call(self, X):
        radial_output = self.radial(X)
        return self.linear(radial_output)


In [6]:
# Define RBF network
model = keras.Sequential([
    RBFLayer(num_centers),
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=2,callbacks=callbacks)


2023-03-29 19:52:41.329709: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-29 19:52:41.338313: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-29 19:52:41.338417: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-29 19:52:41.340409: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

Epoch 1/200


2023-03-29 19:52:43.682089: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-03-29 19:52:43.682157: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2023-03-29 19:52:43.682228: E tensorflow/core/profiler/backends/gpu/cupti_error_manager.cc:133] cuptiGetTimestamp: ignored due to a previous error.
2023-03-29 19:52:43.682243: E tensorflow/core/profiler/backends/gpu/cupti_error_manager.cc:184] cuptiSubscribe: ignored due to a previous error.
2023-03-29 19:52:43.682249: E tensorflow/core/profiler/backends/gpu/cupti_error_manager.cc:457] cuptiGetResultString: ignored due to a previous error.
2023-03-29 19:52:43.682254: E tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1715] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error 
2023-03-29 19:52:45.922972: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f3eac0617b0 initialized for platform CU

4/4 - 4s - loss: 1.7186 - accuracy: 0.3333 - 4s/epoch - 894ms/step
Epoch 2/200
4/4 - 0s - loss: 1.6829 - accuracy: 0.3333 - 116ms/epoch - 29ms/step
Epoch 3/200
4/4 - 0s - loss: 1.6511 - accuracy: 0.3333 - 111ms/epoch - 28ms/step
Epoch 4/200
4/4 - 0s - loss: 1.6182 - accuracy: 0.3333 - 110ms/epoch - 27ms/step
Epoch 5/200
4/4 - 0s - loss: 1.5862 - accuracy: 0.3333 - 111ms/epoch - 28ms/step
Epoch 6/200
4/4 - 0s - loss: 1.5547 - accuracy: 0.3333 - 122ms/epoch - 31ms/step
Epoch 7/200
4/4 - 0s - loss: 1.5241 - accuracy: 0.3333 - 126ms/epoch - 31ms/step
Epoch 8/200
4/4 - 0s - loss: 1.4945 - accuracy: 0.3333 - 128ms/epoch - 32ms/step
Epoch 9/200
4/4 - 0s - loss: 1.4654 - accuracy: 0.3333 - 125ms/epoch - 31ms/step
Epoch 10/200
4/4 - 0s - loss: 1.4364 - accuracy: 0.3333 - 127ms/epoch - 32ms/step
Epoch 11/200
4/4 - 0s - loss: 1.4085 - accuracy: 0.3333 - 128ms/epoch - 32ms/step
Epoch 12/200
4/4 - 0s - loss: 1.3811 - accuracy: 0.3333 - 133ms/epoch - 33ms/step
Epoch 13/200
4/4 - 0s - loss: 1.3547 - 

In [7]:
# Evaluate model on test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")


Test Accuracy: 0.6333


In [10]:
%load_ext tensorboard
# %reload_ext tensorboard
%tensorboard --logdir={log_folder}